# Build complete Dataframe

Our first goal is to obtain the complete Dataframe of a building, that is, getting a time series starting from the first existing hour in the database, and finishing with the last one. Whenever the information in between is missing, we'll fill it with `NaN`, for later processing.

#### Directory structure

./<br></br>
notebook/<br></br>
    &emsp;|--- data-preprocessing<br></br>
    &emsp;&emsp;&emsp;&emsp;|--- complete_dataframe.ipynb<br></br>
out/<br></br>
    &emsp;|--- byday/<br></br>
    &emsp;|--- byhour/

In [18]:
OUT_PATH = 'C:/Users/thmas/OneDrive - Universidad de Castilla-La Mancha/Informática/TFG/out/'

In [19]:
import pandas as pd
import numpy as np
import pymongo as pm
import datetime

In [20]:
HOST = '161.67.142.141'
PORT = 27017
DB = 'differential_uclm_db'
DB_COUNTERRAW = 'CounterRawConsumption'

START_DAY = 5 # Day starts at 5:00 am

### Database connection

In [21]:
def connectDB() -> pm.MongoClient:
    return pm.MongoClient(host=HOST, port=PORT)[DB]

In [22]:
db = connectDB()

## 1. Create hour index

First, we must obtain the building's first and last registered hour, building then the hour index between these dates.

### First and last registered hours
Find first and last registered hours for the specified building ID

In [30]:
def firstHour(db: pm.MongoClient, counter_id: int) -> datetime.datetime:
    return list(db[DB_COUNTERRAW].find({'counterinfo_id': counter_id}).sort('timestamp', pm.ASCENDING).limit(1))[0]['timestamp']

def lastHour(db: pm.MongoClient, counter_id: int) -> datetime.datetime:
    return list(db[DB_COUNTERRAW].find({'counterinfo_id': counter_id}).sort('timestamp', pm.DESCENDING).limit(1))[0]['timestamp']

In [31]:
counter_id = 27 # Building ID example
start, end = firstHour(db, counter_id).replace(hour=5), lastHour(db, counter_id).replace(hour=4) # Fix hours to have 24h days

start, end

(datetime.datetime(2011, 7, 26, 5, 0), datetime.datetime(2020, 2, 13, 4, 0))

### Build hour index
From firstHour lastHour with 1 hour step

In [25]:
def createIndex(first: datetime.datetime, last: datetime.datetime) -> pd.DatetimeIndex:
    return pd.date_range(start=first, end=last, freq='1H')

In [26]:
index = createIndex(firstHour, lastHour)

index

DatetimeIndex(['2011-07-26 05:00:00', '2011-07-26 06:00:00',
               '2011-07-26 07:00:00', '2011-07-26 08:00:00',
               '2011-07-26 09:00:00', '2011-07-26 10:00:00',
               '2011-07-26 11:00:00', '2011-07-26 12:00:00',
               '2011-07-26 13:00:00', '2011-07-26 14:00:00',
               ...
               '2020-02-12 19:00:00', '2020-02-12 20:00:00',
               '2020-02-12 21:00:00', '2020-02-12 22:00:00',
               '2020-02-12 23:00:00', '2020-02-13 00:00:00',
               '2020-02-13 01:00:00', '2020-02-13 02:00:00',
               '2020-02-13 03:00:00', '2020-02-13 04:00:00'],
              dtype='datetime64[ns]', length=74976, freq='H')

## 2. Build complete Dataframe
Now we rebuild complete Dataframe with the consumptions for every hour we got in the index, filling with `NaN` when the value is not found on the database or if it is a negative consumption. This is reindexing the Dataframe with the previous index we obtained

In [11]:
def getDataFrame(db: pm.MongoClient, counter_id: int) -> pd.DataFrame:
    cursor = db[DB_COUNTERRAW].find({'counterinfo_id': counter_id})
    df = pd.DataFrame(list(cursor))
    del df['_id']
    del df['counterinfo_id']
    
    df = df.set_index('timestamp') # Indexing dataframe by timestamp
    
    return df

In [14]:
df = getDataFrame(db, counter_id)
df

,consumption
timestamp,
2011-07-19 14:00:00,0.000000
2011-07-19 15:00:00,0.000000
2011-07-19 16:00:00,0.000000
2011-07-19 17:00:00,0.000000
2011-07-19 18:00:00,0.000000
...,...
2019-04-12 10:00:00,149.737029
2019-04-12 11:00:00,159.370085
2019-04-12 12:00:00,152.738496


### Reindex Dataframe

In [15]:
df = df.reindex(index=index)
df

,consumption
2011-07-19 05:00:00,NaN
2011-07-19 06:00:00,NaN
2011-07-19 07:00:00,NaN
2011-07-19 08:00:00,NaN
2011-07-19 09:00:00,NaN
...,...
2019-04-12 00:00:00,26.667192
2019-04-12 01:00:00,26.660649
2019-04-12 02:00:00,26.663488
2019-04-12 03:00:00,26.673709


### Calculate day
Day recalculation needed because days will start, as defined in `START_DAY`, at 5:00 am

In [16]:
def calcDay(df: pd.DataFrame) -> pd.DataFrame:
    df['day'] = df.apply(lambda x: (x.name - pd.DateOffset(hours=START_DAY)).date(), axis= 1)
    df['day'] = pd.to_datetime(df['day'])
    
    return df

In [17]:
df = calcDay(df)
df

,consumption,day
2011-07-19 05:00:00,NaN,2011-07-19
2011-07-19 06:00:00,NaN,2011-07-19
2011-07-19 07:00:00,NaN,2011-07-19
2011-07-19 08:00:00,NaN,2011-07-19
2011-07-19 09:00:00,NaN,2011-07-19
...,...,...
2019-04-12 00:00:00,26.667192,2019-04-11
2019-04-12 01:00:00,26.660649,2019-04-11
2019-04-12 02:00:00,26.663488,2019-04-11
2019-04-12 03:00:00,26.673709,2019-04-11


## 3. Reshape Dataframe into TimeSeries
Get new Dataframe with indexed with `day`, and its 24 consumptions

In [17]:
consumption = np.asarray(df['consumption'])
consumption = consumption.reshape((len(df['day']) // 24, 24)) # Reshape each day with its 24 consumptions

consumptions = pd.DataFrame(consumption, index=np.arange(len(df['day']) // 24), columns=np.arange(24))
consumptions

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819,17.341348,17.336417,15.108195,13.671267,13.674152,13.431865,12.999662,13.000957,13.004914,12.903182,...,13.242214,15.671428,18.822843,17.765515,17.340447,16.753255,16.913813,17.343652,16.333292,17.337501
2820,22.000545,56.612431,103.271267,145.560168,169.615259,184.831495,188.087845,184.384277,179.326536,162.088170,...,133.884709,110.235443,44.845136,17.856870,15.672561,16.663765,16.662917,16.611020,16.335881,16.330015
2821,21.138057,39.147791,74.582498,138.245750,173.078648,185.526796,191.966432,185.262229,191.044624,170.552954,...,146.308853,108.545867,34.622832,29.404141,26.664616,26.651540,26.676399,25.667485,25.675647,26.380449
2822,29.819443,48.223346,78.543927,143.127575,165.874570,186.572231,189.509033,189.727863,193.731754,186.806203,...,139.478691,107.608301,30.652172,22.393958,20.341397,18.332877,18.340488,17.346131,17.332979,18.335978


### Index by day

In [18]:
days = pd.DataFrame({'day': df['day'].drop_duplicates().tolist()})

consumptions = pd.concat([days, consumptions], axis=1)
consumptions = consumptions.set_index(['day'])

consumptions

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
2011-07-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-07-20,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-07-21,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-07-22,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-07-23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-07,17.341348,17.336417,15.108195,13.671267,13.674152,13.431865,12.999662,13.000957,13.004914,12.903182,...,13.242214,15.671428,18.822843,17.765515,17.340447,16.753255,16.913813,17.343652,16.333292,17.337501
2019-04-08,22.000545,56.612431,103.271267,145.560168,169.615259,184.831495,188.087845,184.384277,179.326536,162.088170,...,133.884709,110.235443,44.845136,17.856870,15.672561,16.663765,16.662917,16.611020,16.335881,16.330015
2019-04-09,21.138057,39.147791,74.582498,138.245750,173.078648,185.526796,191.966432,185.262229,191.044624,170.552954,...,146.308853,108.545867,34.622832,29.404141,26.664616,26.651540,26.676399,25.667485,25.675647,26.380449


In [21]:
consumptions.to_pickle(OUT_PATH + '/consumptions_byday/counter_' + str(counter_id) + '_byDay.zip', compression='zip')